In [ ]:
from jikanpy import Jikan
import json
import time
import pandas as pd

# Initialize Jikan with the v4 base URL
jikan = Jikan(selected_base='https://api.jikan.moe/v4')

In [ ]:
try:
    for page in range(1, 37):
        response = jikan.search('anime', '', page=page, parameters={
            'start_date': '2014-01-01',
            'end_date': '2023-12-31',
            'type': 'tv',
            'min_score': '7'
        })
        print(response)  # Print the raw response to understand what's being returned
        
        # Rate limiting - Wait for 1 second before making the next request
        time.sleep(1)
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
response.keys()

In [ ]:
response['data']

In [ ]:
print(json.dumps(response['data'], indent=4))